# Chapter 15 Threads, Locks

from Gayle Laakmann McDowell's "Cracking the Coding Interview", 6th ed.

Ron Wu

## 15.1 thread vs process

In [1]:
%%file main.java

import java.io.File;
import java.io.IOException;

public class main {

    public static void main(String[] args)  {

        File instFolder = new File(System.getProperty("user.dir") );

        // no share memory
        ProcessBuilder pb = new ProcessBuilder("open", instFolder.toString());
        Process p;
        try {
            p = pb.start();

        } catch (IOException e) {

            e.printStackTrace();
        } 

        someThread t1 = new someThread("t1", 1000);
        someThread t2 = new someThread("t2", 500);
        t1.start();
        t2.start();
    }  
    
    
}

class someThread extends Thread {
    public static int count = 0 ;
    public String name;
    public int sleep;
    public someThread(String na, int sl){
        name=na;
        sleep = sl;
    }
    public void run() {    //add synchronized, then output will be continuum, 0,1,2,3,...
        try{
            for (int i=0 ; i<5; ++i){
                Thread.sleep(sleep); 
                System.out.println(name + " at "+ count);  // reading writiing at the same time
                count += 1;                                // the static count is accessible by both threads                        
            } 
        } catch (InterruptedException e) { 
            e.printStackTrace(); 
        }  
    }
}


Overwriting main.java


In [2]:
! javac main.java
! java main

t2 at 0
t1 at 1
t2 at 2
t2 at 3
t1 at 4
t2 at 5
t2 at 6
t1 at 7
t1 at 8
t1 at 9


## 15.2 context switch

In [3]:
%%file main.java


// it takes time to start another process; it is not free. i.e. parallelism comes with overhead.

// In the following program, the parallel sum only outperforms sequential sum when
// the array size close to half billion, which is in the order of Integer.MAX_VALUE = 2 billion
 

public class main {

    public static void main(String[] args) {

        int size = 50000000;
        int[] a = new int[size] ;
        for (int i = 0; i < size; ++i) {
            a[i] = i;
        }
         
        int steadystate = 5;
        long t0 = System.currentTimeMillis();  
        long sum = 0; 
        for (int j= 0 ; j< steadystate; ++j){ 
            for (int i = 0; i < size; ++i) {
                sum  += a[i];
            }
        }
        System.out.println("Sequenti takes "+ (System.currentTimeMillis()-t0)/steadystate + " millisec, and the sum is "+ sum/steadystate);
        
        System.gc();
        
        sum = 0;
        t0 = System.currentTimeMillis(); 
        
        for (int j= 0 ; j< steadystate; ++j){ 
            someThread t1 = new someThread(a, 0, size/2);
            someThread t2 = new someThread(a, size/2, size);
            t1.start();
            t2.start();
            try {
                t1.join();
                t2.join();
            } catch (InterruptedException e) { 
                e.printStackTrace();
            }
            sum = t1.count + t2.count;
        } 
        System.out.println("Parallel takes "+ (System.currentTimeMillis()-t0)/steadystate + " millisec, and the sum is "+ sum);
  
    }  


}

class someThread extends Thread {
    public long count = 0 ; 
    private int[] a;
    private int l;
    private int r;
    
    public someThread(int[] arr, int left, int right){
        a = arr;
        l = left;
        r = right;
    }

    public void run() { 
        for (int i = l; i < r; ++i) {
            count  += a[i];
        } 
    }
}




Overwriting main.java


In [4]:
! javac main.java
! java main

Sequenti takes 24 millisec, and the sum is 1249999975000000
Parallel takes 15 millisec, and the sum is 1249999975000000


## 15.3 dining philosopher

In [5]:
%%file main.java


// this program will cause deadlock
// Copy to an ide so it can be terminated

import java.util.concurrent.locks.Lock;
import java.util.concurrent.locks.ReentrantLock;
 

public class main {

    public static void main(String[] args) {

        int size = 10; //number of Phil
        phil[] Phil = new phil[size];
        chopstick[] somechop = new chopstick[size];
        
        for (int i=0; i< size; ++i){  
            somechop[i] = new chopstick(i);  
        }
        
        for (int i=0; i< size; ++i){  
            Phil[i] = new phil(i,somechop[i], somechop[(i+1)%size]); 
        } 
        
        for (int i=0; i< size; ++i){  
            Phil[i].start();  
        }
         
    }    
}

class phil extends Thread {
    private int ID ; 
    private chopstick l;
    private chopstick r;
    
    public phil(int id, chopstick left, chopstick right){
        ID = id;
        l = left;
        r = right;
    }  
    public void run(){
        l.inProcession(ID);
        r.inProcession(ID);
        System.out.println("philosopher " + ID + " has obtained both chopsticks");
    } 
}

class chopstick {
    private int ID ; 
    public chopstick(int id){
        ID = id; 
    } 
    private Lock lock = new ReentrantLock();
    public void inProcession(int PhilID) {
        lock.lock(); //whoever thread calls inProcession, it will be locked forever.
        System.out.println("choostick "+ ID +" belongs to philosopher " + PhilID);
        //lock.unlock(); normally we will put lock.unlock(), after we are done
    }
}


Overwriting main.java


In [6]:
%%file main.java


// solved the deadlock problem by adding a lock flag iamLocked

import java.util.concurrent.locks.Lock;
import java.util.concurrent.locks.ReentrantLock;
 

public class main {

    public static void main(String[] args) {

        int size = 10;  
        phil[] Phil = new phil[size];
        chopstick[] somechop = new chopstick[size];
        
        for (int i=0; i< size; ++i){  
            somechop[i] = new chopstick(i);  
        }

        for (int i=0; i< size; ++i){  
            Phil[i] = new phil(i,somechop[i], somechop[(i+1)%size]); 
        } 

        for (int i=0; i< size; ++i){  
            Phil[i].start();  
        }
         
    }    
}

class phil extends Thread {
    private int ID ; 
    private chopstick l;
    private chopstick r;
    
    public phil(int id, chopstick left, chopstick right){
        ID = id;
        l = left;
        r = right;
    }  
    public void run(){
        Lock lock = new ReentrantLock();
        if (!l.iamLocked){
            l.inProcession(ID); 
            if (!r.iamLocked){
                r.inProcession(ID); 
                System.out.println("philosopher " + ID + " has obtained both chopsticks");
            }
            else {
                System.out.println("philosopher " + ID + " cannot eat");
            }

        }  else {
                System.out.println("philosopher" + ID + " cannot eat");
        }
    } 
}

class chopstick {
    private int ID ; 
    public boolean iamLocked = false;
    public chopstick(int id){
        ID = id; 
    } 
    private Lock lock = new ReentrantLock();
    public void inProcession(int philiD) {
        iamLocked = true;
        lock.lock(); 
        System.out.println("choostick "+ ID +" belongs to philosopher " + philiD); 
    }
}
  

Overwriting main.java


In [7]:
! javac main.java
! java main

choostick 1 belongs to prof 1
choostick 6 belongs to prof 6
choostick 5 belongs to prof 5
choostick 4 belongs to prof 4
choostick 2 belongs to prof 2
Prof 2 cannot eat
choostick 0 belongs to prof 0
Prof 0 cannot eat
choostick 3 belongs to prof 3
choostick 9 belongs to prof 9
Prof 9 cannot eat
Prof 4 cannot eat
choostick 8 belongs to prof 8
Prof 5 cannot eat
choostick 7 belongs to prof 6
Prof 6 has obtained both chopsticks
Prof 7 cannot eat
Prof 1 cannot eat
Prof 8 cannot eat
Prof 3 cannot eat


## 15.4 deadlock class

In [8]:
%%file main.java

// Tyical deadlocks occur in circles. A typical solution is to switch the orders, 
// i.e. unknotted. See knot theory

// this program has deadlock. Donot run here

public class main {

    public static void main(String[] args) {

        bankAccount a = new bankAccount();
        bankAccount b = new bankAccount();

        Parallel p1 = new Parallel(a, b, 100d);    // a puts money to b at the same time
        Parallel p2 = new Parallel(b, a, 50d);     // b puts money to a

        p1.start();
        p2.start(); 
    }    
}

class Parallel extends Thread{
    private bankAccount from, to; 
    private double amount;

    public Parallel(bankAccount a, bankAccount b, double d){
        from = a;
        to = b;
        amount = d; 
    }

    public void run(){ 
        from.transfer(to, amount); 
    }
}

class bankAccount {
    
    public double balance = 0d;

    public synchronized void deposit(double amount){
        balance += amount;
    }
    public synchronized void transfer(bankAccount to, double amount) {
        try {
            Thread.sleep(10);
        } catch (InterruptedException e) { 
            e.printStackTrace();
        }
        this.deposit(-amount);
        to.deposit(amount);
    }
}

Overwriting main.java


In [9]:
%%file main.java

// to unknotted it, i took the transfoer function out of the class


public class main {

    public static void main(String[] args) throws InterruptedException {

        bankAccount a = new bankAccount();
        bankAccount b = new bankAccount();

        Parallel p1 = new Parallel(a, b, 100d);
        Parallel p2 = new Parallel(b, a, 50d);

        p1.start();
        p2.start();

        p1.join();
        p2.join();
        
        System.out.println("account a balance: "+ a.balance);
        System.out.println("account b balance: "+ b.balance);
    }    
}

class Parallel extends Thread {
    private bankAccount from, to; 
    private double amount;

    public Parallel(bankAccount a, bankAccount b, double d){
        from = a;
        to = b;
        amount = d; 
    }

    public void run(){ 
        transfer(from, to, amount); 
    }

    public synchronized void transfer(bankAccount from, bankAccount to, double amount)  {
        try {
            Thread.sleep(100);
        } catch (InterruptedException e) { 
            e.printStackTrace();
        }
        from.deposit(-amount);
        to.deposit(amount);
    }
}

class bankAccount {
    
    public double balance = 0d;

    public synchronized void deposit(double amount){
        balance += amount;
    }

}

Overwriting main.java


In [10]:
! javac main.java
! java main

account a balance: -50.0
account b balance: 50.0


## 15.5 call in order

In [11]:
%%file main.java
 

public class main {

    public static void main(String[] args) throws InterruptedException {

        Foo f = new Foo();
        Parallel p1 = new Parallel(f, 1);
        Parallel p2 = new Parallel(f, 2);
        Parallel p3 = new Parallel(f, 3);

        p1.start();
        p2.start();
        p3.start();

    }    
}

class Parallel extends Thread {
    private Foo f;
    private int i;

    public Parallel(Foo f, int i){
        this.f = f;
        this.i = i;
    }

    public void run(){ 
        if (i==1){
            f.first();
        }else if (i==2){
            f.second();
        }else{
            f.third();
        }
    }

}

class Foo {
    public void first() {
        try {
            Thread.sleep(100);
        } catch (InterruptedException e) { 
            e.printStackTrace();
        }
        System.out.println("first");
    }
    public void second() {
        try {
            Thread.sleep(10);
        } catch (InterruptedException e) { 
            e.printStackTrace();
        }
        System.out.println("second");
    }
    public void third() {
        try {
            Thread.sleep(50);
        } catch (InterruptedException e) { 
            e.printStackTrace();
        }
        System.out.println("third");
    }
}

// this will not assure first, then second, then third

Overwriting main.java


In [12]:
! javac main.java
! java main

second
third
first


In [13]:
%%file main.java
 
 
// we need some insurance, use Semaphore

import java.util.concurrent.Semaphore;
import java.util.concurrent.locks.ReentrantLock;

public class main {

    public static void main(String[] args) throws InterruptedException {

        Foo f = new Foo();
        Parallel p1 = new Parallel(f, 1);
        Parallel p2 = new Parallel(f, 2);
        Parallel p3 = new Parallel(f, 3);

        p1.start();
        p2.start();
        p3.start();

    }    
}

class Parallel extends Thread {
    private Foo f;
    private int i;

    public Parallel(Foo f, int i){
        this.f = f;
        this.i = i;
    }

    public void run(){ 
        try{
            if (i==1){
                Thread.sleep(100);
                f.first();
            }else if (i==2){
                Thread.sleep(50);
                f.second();
            }else{
                Thread.sleep(10);
                f.third();
            }
        } catch (InterruptedException ex){

        }
    }

}

class Foo {
    private Semaphore loc1 = new Semaphore(0);  //release 0 permit 
    private Semaphore loc2 = new Semaphore(0);

     
    public void first()  throws InterruptedException{ 


        System.out.println("first"); 
        loc1.release();  //release 1 permit. Whoever needs it can consume it

    }
    public void second() throws InterruptedException{  

        loc1.acquire();  // consume one loc1 permit
        loc1.release();  // release one loc1 permit, so it can continue
        System.out.println("second");
        loc2.release(); 

    }
    public void third() throws InterruptedException{ 

        loc1.acquire(); 
        loc1.release();
        loc2.acquire(); 
        loc2.release(); 
        System.out.println("third");  
    }
}



Overwriting main.java


In [14]:
! javac main.java
! java main

first
second
third


## 15.6 synchronized methods

## 15.7 fizzBuzz